In [ ]:
import datetime

class Bank:
    def __init__(self):
        self.customers = []
        self.employees = []
        self.applications = []

    def add_customer(self, customer):
        self.customers.append(customer)

    def add_employee(self, employee):
        self.employees.append(employee)

    def apply_for_account(self, customer, account_type, initial_deposit=0):
        application = AccountApplication(customer, account_type, initial_deposit)
        self.applications.append(application)
        return f"Application received. Awaiting approval for {customer.name}'s {account_type} account."

    def approve_account_application(self, application):
        if application.customer in self.customers:
            account = application.create_account()
            application.customer.add_account(account)
            return f"Account approved for {application.customer.name}. Account number: {account.account_number}"
        else:
            return "Customer not found. Application rejected."

class Customer:
    def __init__(self, customer_id, name, address, contact):
        self.customer_id = customer_id
        self.name = name
        self.address = address
        self.contact = contact
        self.accounts = []

    def add_account(self, account):
        self.accounts.append(account)

    def get_customer_info(self):
        return f"Customer ID: {self.customer_id}\nName: {self.name}\nAddress: {self.address}\nContact: {self.contact}"

class Account:
    def __init__(self, account_number, balance=0):
        self.account_number = account_number
        self.balance = balance
        self.transactions = []

    def deposit(self, amount):
        if amount > 0:
            self.balance += amount
            self.transactions.append(Transaction("Deposit", amount))
            return f"Deposit successful. New balance: ${self.balance}"
        else:
            return "Invalid deposit amount."

    def withdraw(self, amount):
        if 0 < amount <= self.balance:
            self.balance -= amount
            self.transactions.append(Transaction("Withdrawal", amount))
            return f"Withdrawal successful. New balance: ${self.balance}"
        else:
            return "Invalid withdrawal amount or insufficient funds."

    def transfer(self, target_account, amount):
        if 0 < amount <= self.balance:
            self.balance -= amount
            self.transactions.append(Transaction(f"Transfer to {target_account.account_number}", amount))
            target_account.balance += amount
            target_account.transactions.append(Transaction(f"Transfer from {self.account_number}", amount))
            return f"Transfer successful. New balance: ${self.balance}"
        else:
            return "Invalid transfer amount or insufficient funds."

    def get_account_info(self):
        return f"Account Number: {self.account_number}\nBalance: ${self.balance}"

class Transaction:
    def __init__(self, description, amount):
        self.timestamp = datetime.datetime.now()
        self.description = description
        self.amount = amount

    def __str__(self):
        return f"{self.timestamp}: {self.description} ${self.amount}"

class AccountApplication:
    def __init__(self, customer, account_type, initial_deposit):
        self.customer = customer
        self.account_type = account_type
        self.initial_deposit = initial_deposit
        self.status = "Pending"
        self.approved_account = None

    def create_account(self):
        if self.account_type == "Savings":
            account = SavingsAccount(self.customer, self.initial_deposit)
        elif self.account_type == "Checking":
            account = CheckingAccount(self.customer, self.initial_deposit)
        elif self.account_type == "FixedDeposit":
            account = FixedDepositAccount(self.customer, self.initial_deposit)
        elif self.account_type == "Business":
            account = BusinessAccount(self.customer, self.initial_deposit)
        else:
            raise ValueError("Invalid account type.")
        
        self.status = "Approved"
        self.approved_account = account
        return account

class SavingsAccount(Account):
    def __init__(self, customer, initial_deposit=0, interest_rate=0.02):
        super().__init__(account_number=self.generate_account_number(), balance=initial_deposit)
        self.customer = customer
        self.interest_rate = interest_rate
        self.transactions.append(Transaction("Account opened", initial_deposit))

    def generate_account_number(self):
        return f"S{len(str(len(self.transactions))) * 3}-{len(self.transactions)}"

    def calculate_interest(self):
        interest = self.balance * self.interest_rate
        self.deposit(interest)
        return f"Interest calculated and deposited: ${interest}"

class CheckingAccount(Account):
    def __init__(self, customer, initial_deposit=0, overdraft_limit=1000):
        super().__init__(account_number=self.generate_account_number(), balance=initial_deposit)
        self.customer = customer
        self.overdraft_limit = overdraft_limit
        self.transactions.append(Transaction("Account opened", initial_deposit))

    def generate_account_number(self):
        return f"C{len(str(len(self.transactions))) * 3}-{len(self.transactions)}"

    def withdraw(self, amount):
        if 0 < amount <= self.balance + self.overdraft_limit:
            self.balance -= amount
            self.transactions.append(Transaction("Withdrawal", amount))
            return f"Withdrawal successful. New balance: ${self.balance}"
        else:
            return "Invalid withdrawal amount or overdraft limit exceeded."

class FixedDepositAccount(Account):
    def __init__(self, customer, initial_deposit=0, term=12, interest_rate=0.03):
        super().__init__(account_number=self.generate_account_number(), balance=initial_deposit)
        self.customer = customer
        self.term = term
        self.interest_rate = interest_rate
        self.transactions.append(Transaction("Account opened", initial_deposit))

    def generate_account_number(self):
        return f"FD{len(str(len(self.transactions))) * 3}-{len(self.transactions)}"

    def calculate_interest(self):
        if self.term > 0:
            interest = self.balance * self.interest_rate / 12
            self.deposit(interest)
            self.term -= 1
            return f"Interest calculated and deposited: ${interest}. {self.term} months remaining."
        else:
            return "Fixed deposit matured. Interest calculation complete."

class BusinessAccount(Account):
    def __init__(self, customer, initial_deposit=0, overdraft_limit=5000):
        super().__init__(account_number=self.generate_account_number(), balance=initial_deposit)
        self.customer = customer
        self.overdraft_limit = overdraft_limit
        self.transactions.append(Transaction("Account opened", initial_deposit))

    def generate_account_number(self):
        return f"B{len(str(len(self.transactions))) * 3}-{len(self.transactions)}"

    def withdraw(self, amount):
        if 0 < amount <= self.balance + self.overdraft_limit:
            self.balance -= amount
            self.transactions.append(Transaction("Withdrawal", amount))
            return f"Withdrawal successful. New balance: ${self.balance}"
        else:
            return "Invalid withdrawal amount or overdraft limit exceeded."

class LoanApplication:
    def __init__(self, customer, amount, interest_rate, term):
        self.customer = customer
        self.amount = amount
        self.interest_rate = interest_rate
        self.term = term
        self.status = "Pending"
        self.approved_loan = None

    def apply_for_loan(self):
        if self.customer in bank.customers:
            loan = Loan(self.customer, self.amount, self.interest_rate, self.term)
            self.status = "Approved"
            self.approved_loan = loan
            return loan
        else:
            return "Customer not found. Loan application rejected."

class Loan:
    def __init__(self, customer, amount, interest_rate, term):
        self.customer = customer
        self.amount = amount
        self.interest_rate = interest_rate
        self.term = term
        self.remaining_payments = term
        self.monthly_payment = self.calculate_monthly_payment()

    def calculate_monthly_payment(self):
        monthly_interest_rate = self.interest_rate / 12 / 100
        return (self.amount * monthly_interest_rate) / (1 - (1 + monthly_interest_rate) ** -self.term)

    def make_payment(self):
        if self.remaining_payments > 0:
            self.remaining_payments -= 1
            return f"Payment successful. {self.remaining_payments} payments remaining."
        else:
            return "Loan already paid off."

# Example Usage:
bank = Bank()

bank_manager = BankManager("Alice")

customer1 = Customer(1, "John Doe", "123 Main St", "555-1234")
customer2 = Customer(2, "Jane Doe", "456 Oak St", "555-5678")

savings_application = bank.apply_for_account(customer1, "Savings", 1000)
checking_application = bank.apply_for_account(customer2, "Checking", 500)
fd_application = bank.apply_for_account(customer1, "FixedDeposit", 5000)
business_application = bank.apply_for_account(customer2, "Business", 2000)

print(savings_application)
print(checking_application)
print(fd_application)
print(business_application)

bank_manager.approve_account_application(bank.applications[0])
bank_manager.approve_account_application(bank.applications[1])
bank_manager.approve_account_application(bank.applications[2])
bank_manager.approve_account_application(bank.applications[3])

print(customer1.get_customer_info())
print(customer1.accounts[0].get_account_info())
print(customer1.accounts[0].calculate_interest())
print(customer1.accounts[1].get_account_info())

print(customer2.get_customer_info())
print(customer2.accounts[0].get_account_info())
print(customer2.accounts[0].withdraw(600))
print(customer2.accounts[1].get_account_info())

loan_application = LoanApplication(customer1, 5000, 8, 24)
approved_loan = bank_manager.approve_loan(loan_application)

print(loan_application.apply_for_loan())
print(approved_loan.make_payment())
print(approved_loan.make_payment())
